This notebook walks through how to create world files for images pulled by WKT using the ExtractOrthoImage API call. It will generate world files for each of the Polygons in a WKT txt file.

We will be using mostly universal libraries to do this so it can easily be converted to additional languages. The only exception is the Pillow library that we will use to get the image file's width and height.

First we set up a function that will be used lated to determine the pixel width and height of each image in degrees.

In [10]:
import requests
import json
import urllib.request
import re
from PIL import Image

def degrees_per_pixel(bounds, pixels_wide, pixels_high):
    width_in_deg = bounds[2] - bounds[0]
    height_in_deg = bounds[3] - bounds[1]

    deg_per_pixel_wide = width_in_deg/pixels_wide
    deg_per_pixel_high = height_in_deg/pixels_high
    return deg_per_pixel_wide, deg_per_pixel_high

Generate a token for use with the API call to get the ortho images

In [11]:
with open("credentials.json") as fp:
    credentials = json.load(fp) # {'password': 'yourpasswordhere', 'username': 'email@address.com'}
    
r = requests.post("https://api.gic.org/auth/Login/", data=credentials)
token = r.json()['token']

Use the ExtractOrthoImages API call to retrieve ortho images based on WKTs for your AOIs. 

After the images are downloaded, open them up and store the width and height of each for use later.

In [12]:
extractOrthoURL = "https://api.gic.org/images/ExtractOrthoImages"

layer = "bluesky-ultra"
wkts = []
readWKT = open("wkt.txt","r")
imageWidths = []
imageHeights = []

for iterate,line in enumerate(readWKT):
    stripped = line.strip()
    start_url = extractOrthoURL+"?layer="+layer+"&epsg=4326&wkt="+stripped+"&AuthToken="+token
    url = start_url.replace(" ","%20")
    urllib.request.urlretrieve(url, "Image"+str(iterate)+".jpg")
    wkts.append(stripped)
    #Grab the image width and height of the file after downloading and store for use
    image = Image.open("Image"+str(iterate)+".jpg")
    imageWidths.append(image.size[0])
    imageHeights.append(image.size[1])

Strip the WKTs down to just latitude and longitude combinations. We will use these to determine the bounding box of the image that is downloaded using ExtractOrthoImages.

In [13]:
wktValue = []

for wkt in wkts:
    adjustment = wkt.replace("POLYGON((", "")
    stripped = adjustment.replace("))","")
    values = re.split("\s|,", stripped)
    wktValue.append(values)

Iterate through each of the latitude and longitudes to determine the maximum and minimum of each WKT. 

**In this notebook since the loop is broken into multiple cell for explanatory purposes, this will only run the first iteration. If you skip over these you can run the full loop in the cell in the end.

In [14]:
tempMinLat = 0.0
tempMinLong = 0.0
tempMaxLat = 0.0
tempMaxLong = 0.0
counter = 0

for each in wktValue:
    tempMinLong = float(each[0])
    tempMinLat = float(each[1])
    tempMaxLong = float(each[0])
    tempMaxLat = float(each[1])
    for i in range(0, len(each), 2):
        if float(each[i]) < tempMinLong:
            tempMinLong = float(each[i])
        if float(each[i]) > tempMaxLong:
            tempMaxLong = float(each[i])
    for i in range(1,len(each), 2):
        if float(each[i]) < tempMinLat:
            tempMinLat = float(each[i])
        if float(each[i]) > tempMaxLat:
            tempMaxLat = float(each[i])

Store the bounds to be used with the function to determine the degrees per pixel of the width and height.

The southwest corner is the minimum latitude and minimum longitude.

The northeast corner is the maximum latitude and maximum longitude.

In [15]:
    sw = float(tempMinLong), float(tempMinLat)
    ne = float(tempMaxLong), float(tempMaxLat)

    bounds = [*sw, *ne]

Use the function to determine and store the pixel width and height in degrees

In [16]:
    pixWidth, pixHeight = degrees_per_pixel(bounds, imageWidths[counter], imageHeights[counter])

Get the coordinates of the top left pixel for the world file. The top left corner is the maximum latitude and minimum longitude.

Also format the width and height per pixel for use with the world file.

In [17]:
    topLeftX = tempMaxLat
    topLeftY = tempMinLong

    pixw = f"{pixWidth:.12f}"
    pixh = f"{pixHeight:.12f}"

Open a file to create the JPW or JGW that fits the image and write the contents to the file.

The first line is the width per pixel in degrees.

The second and third line are rotation along the y-axis and x-axis - While using NADIR these will likely always be 0.

The fourth line is the height per pixel in degrees.

The fifth line is the Y coordinate of the top left pixel.

The sixth line is the X coordinate of the top left pixel.

In [18]:
    f = open("Image"+str(counter)+".jgw", "w")
    f.write(pixw+"\n")
    f.write("0\n")
    f.write("0\n")
    f.write("-"+pixh+"\n")
    f.write(str(topLeftY)+"\n")
    f.write(str(topLeftX)+"\n")

    f.close()

    counter += 1

**This is the cell that contains the full loop to run if you want to run multiple iterations

In [ ]:
tempMinLat = 0.0
tempMinLong = 0.0
tempMaxLat = 0.0
tempMaxLong = 0.0
counter = 0

for each in wktValue:
    tempMinLong = float(each[0])
    tempMinLat = float(each[1])
    tempMaxLong = float(each[0])
    tempMaxLat = float(each[1])
    for i in range(0, len(each), 2):
        if float(each[i]) < tempMinLong:
            tempMinLong = float(each[i])
        if float(each[i]) > tempMaxLong:
            tempMaxLong = float(each[i])
    for i in range(1,len(each), 2):
        if float(each[i]) < tempMinLat:
            tempMinLat = float(each[i])
        if float(each[i]) > tempMaxLat:
            tempMaxLat = float(each[i])

    
    sw = float(tempMinLong), float(tempMinLat)
    ne = float(tempMaxLong), float(tempMaxLat)

    bounds = [*sw, *ne]

    pixWidth, pixHeight = degrees_per_pixel(bounds, imageWidths[counter], imageHeights[counter])

    topLeftX = tempMaxLat
    topLeftY = tempMinLong

    pixw = f"{pixWidth:.12f}"
    pixh = f"{pixHeight:.12f}"

    f = open("Image"+str(counter)+".jgw", "w")

    f.write(pixw+"\n")
    f.write("0\n")
    f.write("0\n")
    f.write("-"+pixh+"\n")
    f.write(str(topLeftY)+"\n")
    f.write(str(topLeftX)+"\n")

    f.close()

    counter += 1